

# **Data Science, 2024**



A demonstration of exploratory data analysis to accompany the Lecture.

In [ ]:
import numpy as np
import pandas as pd

These options are used to customize the display of pandas DataFrames for better readability and precision when working with data.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)


# Tuberculosis in the United States

What can we say about the presence of Tuberculosis in the United States?

Let's look at the data included in the [original CDC article](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down) published in 2021.

**CSV and Nice Field Names**

Suppose Table 1 was saved as a CSV file located in `data/cdc_tuberculosis.csv`. We can then explore the CSV (which is a text
file, and does not contain binary-encoded data) in many ways:
1. Using a text editor like the one in DataHub (right-click on the file and use `Open->Editor`), emacs, vim, VSCode, etc.
2. Opening the CSV directly in DataHub (read-only), Excel, Google Sheets, etc.
3. The Python file object
4. pandas, using `pd.read_csv()`

Let's start with the first two so we solidify the idea of a CSV as **rectangular data (i.e., tabular data) stored as comma-separated values**.

Try 1, 2.

Then Let's see the tried-and-true Data Science approach: pandas.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read tuberculosis data
df=pd.read_csv("/content/drive/MyDrive/DataScience_with_KS/Data Exploratory analysis/cdc_tuberculosis.csv",header=1)
df

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


Wait, what's up with the "Unnamed" column

names? And the first row, for that matter?

Congratulations -- you're ready to wrangle your data. Because of how things are stored, we'll need to clean the data a bit to name our columns better.

A reasonable first step is to identify the row with the right header. The `pd.read_csv()` function ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)) has the convenient `header` parameter:

In [ ]:
df.dtypes

,0
U.S. jurisdiction,object
2019,object
2020,object
2021,object
2019.1,float64
2020.1,float64
2021.1,float64


Wait...but now we can't differentiate betwen the "Number of TB cases" and "TB incidence" year columns. pandas has tried to make our lives easier by automatically adding ".1" to the latter columns, but this doesn't help us as humans understand the data.

We can do this manually with `df.rename()`

In [ ]:
# Rename column List
df=df.rename(columns={"2019":"TB cases 2019","2020":	"TB cases 2020","2021":"TB cases 2021","2019.1":"TB incidence 2019",	"2020.1":"TB incidence 2020"	,"2021.1":"TB incidence 2021"})
df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,"8,900","7,173","7,860",2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


# Record Granularity

You might already be wondering: What's up with that first record?

Row 0 is what we call a **rollup record**, or summary record. It's often useful when displaying tables to humans. The **granularity** of record 0 (Totals) vs the rest of the records (States) is different.


Okay, EDA step two. How was the rollup record aggregated?

Let's check if Total TB cases is the sum of all state TB cases. If we sum over all rows, we should get **2x** the total cases in each of our TB cases by year (why?).

In [ ]:
# Apply sum to tb DataFrame

df.sum()

,0
Unnamed: 0,U.S. jurisdictionTotalAlabamaAlaskaArizonaArka...
No. of TB cases,"20198,9008758183642,11166671824558302997326108..."
Unnamed: 2,"20207,1737258136591,70652541719412221928216923..."
Unnamed: 3,"20217,8609258129691,75058544319499228106425512..."
TB incidence,2128.94
Unnamed: 5,2113.09
Unnamed: 6,2123.94



Whoa, what's going on? Check out the column types:

In [ ]:
# Find datatype for each column
df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       53 non-null     object 
 1   No. of TB cases  53 non-null     object 
 2   Unnamed: 2       53 non-null     object 
 3   Unnamed: 3       53 non-null     object 
 4   TB incidence     53 non-null     float64
 5   Unnamed: 5       53 non-null     float64
 6   Unnamed: 6       53 non-null     float64
dtypes: float64(3), object(4)
memory usage: 3.0+ KB


,0
Unnamed: 0,object
No. of TB cases,object
Unnamed: 2,object
Unnamed: 3,object
TB incidence,float64
Unnamed: 5,float64
Unnamed: 6,float64


Looks like those commas are causing all TB cases to be read as the `object` datatype, or **storage type** (close to the Python string datatype), so pandas is concatenating strings instead of adding integers.

Fortunately `read_csv` also has a `thousands` parameter (for what it's worth, I didn't know this beforehand--I [googled](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) this):

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/DataScience_with_KS/Data Exploratory analysis/cdc_tuberculosis.csv",thousands=",",header=1)
data
# we can remove commas to change the data types as it was and object due to comma.
# But now we already did this using tousand paremeter while loading data

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


In [ ]:
df_2010

,U.S. jurisdiction,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
5,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...
51,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519
52,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893
53,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147
54,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434


In [ ]:
# now apply sum
# Answer Here
data=data.dropna()
data.sum()

,0
U.S. jurisdiction,AlabamaAlaskaArizonaArkansasCaliforniaColorado...
2019,8900
2020,7173
2021,7860
2019.1,107.23
2020.1,90.93
2021.1,100.57


The Total TB cases look right. Phew!

(We'll leave it to your own EDA to figure out how the TB incidence "Totals" were aggregated.)

Let's just look at the records with **state-level granularity**:

In [ ]:
# Answer Here
data=data[data["U.S. jurisdiction"]!="Total"]
data

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
...,...,...,...,...,...,...,...
47,Virginia,191,169,161,2.23,1.96,1.86
48,Washington,221,163,199,2.90,2.11,2.57
49,West Virginia,9,13,7,0.50,0.73,0.39
50,Wisconsin,51,35,66,0.88,0.59,1.12


What do each of these values represent? Why?

To the lecture!


# Gather Census Data

U.S. Census population estimates [source](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) (2019), [source](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html) (2020-2021).

Running the below cells cleans the data. We encourage you to closely explore the CSV and study these lines after lecture...

There are a few new methods here:
* `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
* `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [ ]:
# Load 2010s census data
df_2010=pd.read_csv("/content/drive/MyDrive/DataScience_with_KS/Data Exploratory analysis/nst-est2019-01.csv",thousands=",",header=3)



# Apply some EDA

Drop the column of Estiamte Base

Rename Unnamed: 0 as 'Geographic Area'

"smart" converting of columns(.convert_dtypes()) Google it. Use at your own risk

.dropna() to drop records with NaN

You can also suggest any change that can be helpful for EDA



In [ ]:
# Drop the column of Estiamte Base
df_2010.drop("Estimates Base",axis=1)
df_2010=df_2010.rename(columns={"Unnamed: 0":"Geographic Area"})
df_2010=df_2010.convert_dtypes()  # It will convert the data in most suitable data type
df_2010=df_2010.dropna()



In [ ]:
df_2010=df_2010.drop([0,1,2,3,4,57],axis=0)


In [ ]:
df_2010=df_2010.drop("Estimates Base",axis=1)
df_2010

,Geographic Area,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
5,.Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,.Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,.Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,.Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,.California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...
51,.Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519
52,.Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893
53,.West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147
54,.Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434


In [ ]:
# census 2020s data
df_2020= pd.read_csv("/content/drive/MyDrive/DataScience_with_KS/Data Exploratory analysis/NST-EST2022-POP.csv", header=3,thousands=",")
df_2020

,Unnamed: 0,Unnamed: 1,2020,2021,2022
0,United States,331449520.00,331511512.00,332031554.00,333287557.00
1,Northeast,57609156.00,57448898.00,57259257.00,57040406.00
2,Midwest,68985537.00,68961043.00,68836505.00,68787595.00
3,South,126266262.00,126450613.00,127346029.00,128716192.00
4,West,78588565.00,78650958.00,78589763.00,78743364.00
...,...,...,...,...,...
58,Note: The estimates are developed from a base ...,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN
60,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN


In [ ]:
df_2020=df_2020.drop([0,1,2,3,4,57,58,59,60,61,62],axis=0)


In [ ]:
# perform EDA Here as did on 2010s census data

df_2020=df_2020.rename(columns={"Unnamed: 0":"Geographic Area"})
df_2020=df_2020.convert_dtypes()
df_2020=df_2020.dropna()


In [ ]:
df_2020=df_2020.drop("Unnamed: 1",axis=1)
df_2020

,Geographic Area,2020,2021,2022
5,.Alabama,5031362,5049846,5074296
6,.Alaska,732923,734182,733583
7,.Arizona,7179943,7264877,7359197
8,.Arkansas,3014195,3028122,3045637
9,.California,39501653,39142991,39029342
...,...,...,...,...
51,.Virginia,8636471,8657365,8683619
52,.Washington,7724031,7740745,7785786
53,.West Virginia,1791420,1785526,1775156
54,.Wisconsin,5896271,5880101,5892539





# Join Data (Merge DataFrames)


Time to `merge`! Here I use the DataFrame method `df1.merge(right=df2, ...)` on DataFrame `df1` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)). Contrast this with the function `pd.merge(left=df1, right=df2, ...)` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge)). Feel free to use either.

In [ ]:
df_2010.rename(columns = {'Geographic Area':'U.S. jurisdiction'}, inplace= True)


In [ ]:
df_2010['U.S. jurisdiction'] = df_2020['U.S. jurisdiction'].str.replace('.','')
df_2010

,U.S. jurisdiction,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
5,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,Alaska,710231,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,Arizona,6392017,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,Arkansas,2915918,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,California,37253956,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223
...,...,...,...,...,...,...,...,...,...,...,...,...
51,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519
52,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893
53,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147
54,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434


In [ ]:
df_2020.rename(columns = {'Geographic Area':'U.S. jurisdiction'}, inplace= True)

In [ ]:
df_2020['U.S. jurisdiction'] = df_2020['U.S. jurisdiction'].str.replace('.','')
df_2020

,U.S. jurisdiction,2020,2021,2022
5,Alabama,5031362,5049846,5074296
6,Alaska,732923,734182,733583
7,Arizona,7179943,7264877,7359197
8,Arkansas,3014195,3028122,3045637
9,California,39501653,39142991,39029342
...,...,...,...,...
51,Virginia,8636471,8657365,8683619
52,Washington,7724031,7740745,7785786
53,West Virginia,1791420,1785526,1775156
54,Wisconsin,5896271,5880101,5892539


In [ ]:
# merge TB dataframe with two US census dataframes
tb_census_df = pd.merge(data,df_2010,left_index=True,right_index=True,how="outer")
tb_census_df_final=pd.merge(tb_census_df,df_2020,left_index=True,right_index=True,how="outer")
tb_census_df_final

,U.S. jurisdiction_x,2019_x,2020_x,2021_x,2019.1,2020.1,2021.1,U.S. jurisdiction_y,Census,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019_y,U.S. jurisdiction,2020_y,2021_y,2022
1,Alabama,87.00,72.00,92.00,1.77,1.43,1.83,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,Alaska,58.00,58.00,58.00,7.91,7.92,7.92,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,Arizona,183.00,136.00,129.00,2.51,1.89,1.77,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,Arkansas,64.00,59.00,69.00,2.12,1.96,2.28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,California,2111.00,1706.00,1750.00,5.35,4.32,4.46,Alabama,4779736,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,Alabama,5031362,5049846,5074296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,Wyoming,1.00,0.00,3.00,0.17,0.00,0.52,Virginia,8001024,8023699,8101155,8185080,8252427,8310993,8361808,8410106,8463587,8501286,8535519,Virginia,8636471,8657365,8683619
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington,6724540,6742830,6826627,6897058,6963985,7054655,7163657,7294771,7423362,7523869,7614893,Washington,7724031,7740745,7785786
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Virginia,1852994,1854239,1856301,1856872,1853914,1849489,1842050,1831023,1817004,1804291,1792147,West Virginia,1791420,1785526,1775156
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wisconsin,5686986,5690475,5705288,5719960,5736754,5751525,5760940,5772628,5790186,5807406,5822434,Wisconsin,5896271,5880101,5892539


This is a little unwieldy. We could either drop the unneeded columns now, or just merge on smaller census DataFrames. Let's do the latter.


## Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$$\text{TB incidence} = \frac{\text{# TB cases in population}}{\text{# groups in population}} = \frac{\text{# TB cases in population}}{\text{population}/100000} $$

$$= \frac{\text{# TB cases in population}}{\text{population}} \times 100000$$

Let's try this for 2019:

Awesome!!!

Let's use a for-loop and Python format strings to compute TB incidence for all years. Python f-strings are just used for the purposes of this demo, but they're handy to know when you explore data beyond this course ([Python documentation](https://docs.python.org/3/tutorial/inputoutput.html)).

In [ ]:
# recompute incidence for all years (2019, 2020, 2021)
years = ["2019", "2020", "2021"]
for year in years:
    df[f'T incidence {year}'] = df[f'TB cases {year}'] / df[f'{year}'] * 100000
    print(df[f'TB incidence {year}'])

KeyError: 'TB cases 2019'

In [ ]:
df.columns

These numbers look pretty close!!! There are a few errors in the hundredths place, particularly in 2021. It may be useful to further explore reasons behind this discrepancy. We'll leave it to you!

In [ ]:
df.describe()